## Linear Regression Modeling and an Introduction to sklearn
### Introduction to Data Science
### Last Updated: November 19, 2022
---  

### PREREQUISITES
- variables
- data types
- pandas

### SOURCES 
- [sklearn Introduction](https://scikit-learn.org/stable/index.html)
- [R-Squared in sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html)

### OBJECTIVES
- Identify variables that should not be used as predictors as they would introduce bias into the model
- Understand why a model is trained on a portion of the data
- Introduce some basic functionality of the `sklearn` package
- Illustrate how to fit a linear regression model with `sklearn`
- Compare the fit of two linear regression models using their fit statistics

### CONCEPTS
- `sklearn` interface
- predictors that can introduce bias or ethical issues should not be used in models
- splitting data into training set and testing set
- generalization of patterns to new data
- target variable and predictor variables
- making predictions from a model
- assessing model fit with R-squared

---

## I. Motivating Model Fitting

We now have cleaned, prepared data in a format ready for modeling.  
We learned how to transform data into useful features.  
Here, we discuss fitting a model to the data.  
Typical reasons for modeling:
- understanding how the features **explain** the target
- using the features to **predict** the target

Model fitting is done with software packages.  
We discuss a powerful Python package for machine learning: `scikit-learn` (`sklearn` for short).  

This notebook will provides an outline of *linear regression* and a very brief demo of `sklearn`. You are encouraged to explore and go further.

**NOTE:**  
Be sure to confirm the validity of the data before training a model on it.  
Describing the data with statistical summaries will be helpful here.

## II. Introducing Linear Regression

The simplest model form is linear regression, which assumes a linear relationship between one or more predictors $X$ and expected value of the target $Y$. 

We won't discuss all of the Gauss Markov assumptions of linear regression (but a good book on the subject will cover it).

The mathematical form is:

$E(Y) = \beta_0 + \beta_1 X_1 + ... + \beta_n X_n$

where 

- $E(Y)$ is the target
- $\beta_0$ is an intercept parameter
- $\beta_1$, ..., $\beta_n$ are slope parameters or *weights*
- $X_1$, ..., $X_n$ are predictors
- $n$ is the number of predictors, which is one in the simplest case

When all predictors are zero, the target value is equal to $\beta_0$

For the case of a single predictor, the model is called *simple linear regression*.

**The key to regression is to discover the best set of predictors that explains or predicts the target.**  
As we add useful predictors, it is possible to minimize the error between the predicted values and the target values.

Additionally, the intercept and weights are estimated from data (model fitting).  
These parameter values are determined such that the errors between the predicted values and the target values are minimized.  


We turn to model fitting with sklearn next.

## III. Being Thoughtful About Features

It may be tempting, particularly in the age of automation, to include all features in the model, perhaps letting the algorithm arrive at the best set.

There are some important things to think about when deciding features:

1. The features should make intuitive sense. In practice, the data scientist might discuss the features with the line of business, the product manager, and other stakeholders.

2. **The features should not introduce bias into the model, or introduce ethical issues.**

One needs to be careful around the use of demographic variables, as it can be easy to discriminate based on age, gender, race, and ethnicity.

Examples of variables that would introduce bias, and therefore should be avoided in modeling, include:

- age, race, ethnicity, gender, religion, disability, color, national origin, religion
- employment type, where some jobs are skewed toward one gender, such as nursing (this targets females)
- an indicator if a person pressed 2 to hear a message in Spanish (this targets an ethnicity)
- salary (this targets race/ethnicity)
- the kind of cuisine served at a restaurant (this is more subtle, but can target race/ethnicity)

For the *salary* predictor, if one were predicting who might default on a mortgage, it would be better from an ethics viewpoint to use *debt/income* as a predictor.

---

**THINK ABOUT AND DISCUSS**

Can you think of other predictors that could introduce bias in a model?

---

In summary, the model predictors should be carefully considered. It is not enough that predictors improve the model fit; they must also make sense and not cause harm.

## IV. Fit a Regression Model

### A. Fetch and Inspect Data

import sklearn dataset and functionality

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score # R-squared

import pandas as pd

Next, we fetch a built-in housing price dataset from `sklearn`.  
The data is based on a California district and it is aggregated.

In [ ]:
housing = fetch_california_housing()

In [ ]:
housing

In [ ]:
type(housing)

Inspect what's in the object

In [ ]:
housing.keys()

Look at the target variable:

In [ ]:
housing.target

The target is also called the `response variable`, `outcome variable`, or `dependent variable`.  
Different subject areas give it a different name. In machine learning, `target` is used.

**The targets represent scaled house prices (they are divided by 100,000)**

*housing.data* contains the data in an array object.

In [ ]:
housing.data

the shape of the array: (rows, columns)

In [ ]:
housing.data.shape

Next, look at the feature names. Features are also called `factors`, `predictors`, or `independent variables`.  
Again, this depends on the subject area. In machine learning, `feature` or `predictor` is used.

In [ ]:
housing.feature_names

**These features may be useful in predicting house prices (the targets).**

---

### B. Data Splitting

When we train a model on a dataset, it learns the patterns in that data.    
In fact, it might learn it too well, memorizing nuances (overfitting).  
There is a very real risk that the model might not do well on new data it hasn't seen.

To help reduce this risk, we do this:  
1. Split the data into several pieces. Here, we use two pieces called a **train set** and a **test set**.  
2. Train the model on the train set  
3. Evaluate the model on the test set

The hope is that the model learns the right patterns, and that it **generalizes** to the test set.  
This means that the model performs similarly on the test set.


Split the data into 60% train, 40% test sets. The helper function `train_test_split` can do this:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, train_size = 0.6, random_state=314)

Show the dataset sizes

In [ ]:
len(x_train), len(x_test), len(y_train), len(y_test)

**Question: what is the data type holding these numbers?**

---

### C. Data Scaling

Next, we will scale the data by following these steps:

1. Scale the x_train and x_test data using `StandardScaler`. **Why do we do this?**
2. Train a regression model on (x_train, y_train). **This is common notation to show that X and Y are paired into observations used in training the model**
3. Feed `x_test` into the model to make predictions `y_test_predicted`
4. Evaluate model performance by comparing `y_test_predicted` to `y_test`

---

**1. Instantiate StandardScaler object and fit it**

In [ ]:
scaler = StandardScaler()
x_train_s = scaler.fit_transform(x_train)

How this works: the scaler does three things:
1. it computes the means and standard deviations of each column and saves them
2. for each column, it subtracts its mean
2. for each column, it divides by its standard deviation

Look at the scaled training data.

In [ ]:
x_train_s

The column means are roughly all zero. This shows they are centered.

In [ ]:
x_train_s.mean(axis=0)

The column standard deviations are all one. This shows they are scaled.

In [ ]:
x_train_s.std(axis=0)

### D. Set up and Fit Linear Regression Model

<img src="linreg.png">

The order of the passed data matters: (x, y)

Remember to used the scaled X data

In [ ]:
reg = LinearRegression().fit(x_train_s, y_train)

show the predictor coefficients (also called `weights` or `betas`)

In [ ]:
reg.coef_

show the `intercept`

In [ ]:
reg.intercept_

**3. Predictions on test set (use *x_test_s* to predict *y_test*).**

First, you'll want to scale the `x_test` data.  
**Important point**: reuse the scaler from the training data.

In [ ]:
x_test_s = scaler.fit_transform(x_test)

Next, feed `x_test_s` into the model to predict `y_test`

In [ ]:
y_test_predicted = reg.predict(x_test_s)

### E. Evaluate Model Performance

Notice: since we know `y_test`, we can compare it to our predictions.  

A common model fit metric for linear regression is `R-squared`.  

It measures the fraction of variation in Y explained by the model. A higher value is better.

The value's range is $[0,1]$ where zero means no explanatory power, and one means perfect explanatory power.

If you are interested in the details of the calculation, see the **Appendix** below.

___

We compute R-squared by calling the function: `r2_score`  
Note that the order of the inputs matters: (true, predicted) 

In [ ]:
print(r2_score(y_test, y_test_predicted))

This model explains more than half of the variation in prices.

## IV. Comparing Models

Next, we build a model using **only the first three predictors**, and measure the R-squared for comparison.  
It probably won't be as high.

In [ ]:
x_train_subset = x_train[:,:3] # use all rows, and first three columns

Create a new scaling object

In [ ]:
scaler2 = StandardScaler()
x_train_subset_s = scaler2.fit_transform(x_train_subset)

Fit a new model

In [ ]:
reg2 = LinearRegression().fit(x_train_subset_s, y_train)

Now transform the first three predictors in the test set:

In [ ]:
x_test_subset_s = scaler2.fit_transform(x_test[:,:3])

Make predictions

In [ ]:
y_test_predicted2 = reg2.predict(x_test_subset_s)

Compute R-squared

In [ ]:
print(r2_score(y_test, y_test_predicted2))

Just as we expected, it doesn't do as well: 0.521 versus 0.608 from the larger model.  
This tells us that the predictors we dropped were important in explaining the target.

**Ultimately, R-squared is a useful way to compare the predictive power to two models.**

---

#### TRY FOR YOURSELF

1. Repeat the process on your own dataset:
   - Use `sklearn` to split the data, select predictors, scale them, fit a model, predict the targets in the test set, and compute R-squared
   
   

2. Select a different set of predictors and repeat the process from (1). What do the results tell you?

---  

## Appendix: Calculating R-Squared with a Formula

`R-squared` is easily calculated in software, but the formula is:

$$R-squared = 1 - SS_{res} / SS_{tot}$$

where

`SS_res` is the sum of squared residuals. This is computed by:
1. calculating the residuals: `y_test - y_test_predicted` for each data point. these are errors, where smaller is better.
2. squaring the residuals
3. summing the squared residuals

`SS_tot` is the total sum of squares. This is computed by:
1. Computing the average of `y_test`, which we call `y_test_mean`
2. calculating the residuals: `y_test - y_test_mean` for each data point.
3. squaring the residuals
4. summing the squared residuals

Conceptually, there is generally variance in the data, and `SS_tot` is proportional to that variance.  
if `SS_{res}` is much smaller than `SS_tot`, it says that the model is explaining most of that variance.  
This results in `R-squared` close to one.
___